In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random

# os 관리를 위한 라이브러리
import os
import gc

In [2]:
# 경고메세지 끄기

import warnings
warnings.filterwarnings(action = 'ignore')

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

In [4]:
seed_everything(42) # Seed 고정

In [5]:
### 전처리된 파일 불러오기

train = pd.read_parquet('./data/train_pre.parquet')
test = pd.read_parquet('./data/test_pre.parquet')
sample_submission = pd.read_csv('./data/sample_submission.csv', index_col = 0)

In [6]:
train.head(3)

,Month,Estimated_Departure_Time,Estimated_Arrival_Time,Origin_Airport_ID,Destination_Airport_ID,Distance,Carrier_ID(DOT),Tail_Number,Delay,Holiday
1,8,460,624,13930,14869,1250.0,20304,310,NaN,0
4,1,540,619,14771,10157,250.0,20304,554,NaN,0
5,4,945,1011,11618,11278,199.0,20452,3430,1.0,0


In [7]:
test.head(3)

,Month,Estimated_Departure_Time,Estimated_Arrival_Time,Origin_Airport_ID,Destination_Airport_ID,Distance,Carrier_ID(DOT),Tail_Number,Holiday
0,12,716,779,12266,14683,191.0,19977,4382,0
1,9,900,1035,11618,10397,746.0,19790,1934,0
2,3,960,1155,13930,12953,733.0,19977,2144,0


# **3. 준지도학습**
- 라벨이 없는 데이터에 대한 라벨링 진행

## **3-0. 데이터 분리**

In [8]:
### 데이터 분리
# 레이블이 있는 데이터와 없는 데이터 분리

train_labeled = train.loc[train['Delay'].notnull(),:]
train_unlabeled = train.loc[train['Delay'].isnull(),:]

In [9]:
train_labeled.shape[0]

172245

In [10]:
train_unlabeled.shape[0]

502770

In [11]:
### X,y 분리

# train
X_labeled = train_labeled.drop(['Delay'],axis = 1)
y_labeled = train_labeled['Delay']
X_unlabeled = train_unlabeled.drop(['Delay'],axis = 1)

print("X Labeled: ", X_labeled.shape)
print("y Labeled: ", y_labeled.shape)
print()
print("X Unlabeled: ", X_unlabeled.shape)

# test
X_test = test
print()
print("X Test: ", X_test.shape)

X Labeled:  (172245, 9)
y Labeled:  (172245,)

X Unlabeled:  (502770, 9)

X Test:  (1000000, 9)


## **3-1. 기본 모델 모델링(Base Model Modeling)**
- Base Model 선정 & 학습
  - 레이블이 **있는** 데이터만을 사용하여 모델을 학습시키면서 튜닝을 진행
- 기본 분류기인 Base Model의 하이퍼 파라미터 튜닝을 수행
  - ```GridSearchCV```, ```RandomizedSearchCV```, 또는 ```Bayesian optimization``` 등의 방법 활용
  - 주어진 데이터와 문제에 맞는 하이퍼파라미터 조합을 찾는 것이 중요

**📌 LogLoss**  
- $- (1/N) * Σ[y_i * log(p_i) + (1 - y_i) * log(1 - p_i)]$
  - N: 데이터 포인트(샘플)의 수
  - y_i: 실제 레이블(이진 분류의 경우 0 또는 1)
  - p_i
    - 모델의 예측값 => 해당 데이터 포인트가 1(= Delayed)에 속할 확률
    - 일반적으로 모델의 ```predict_proba()``` 메서드를 사용하여 얻음
- 각 데이터 포인트에서 실제 레이블과 모델의 예측값 사이의 로그 손실을 계산하고, 이를 모든 데이터 포인트에 대해 평균하여 구함
  - 작을수록 good

### **📚 결과 정리**

|Model|LogLoss|
|-------|-----|
|DecisionTreeClassifier|0.447902|
|RandomForest|0.445446|
|GradientBoosting|0.440778|
|LightGBM|0.439663| 
|XGBoost|0.440057| 

=> Base Model로 **LightGBM**을 채택함

In [12]:
### 최적 파라미터로 튜닝된 모델을 base model로 생성

from lightgbm import LGBMClassifier
base_model = LGBMClassifier(learning_rate = 0.05, max_depth = 7, 
                            n_estimators = 300, num_leaves = 31)

## **3-2. Self Training**
- 초기에 레이블이 없는 데이터를 레이블링하여 일부 레이블이 있는 데이터를 생성한 후, 이 데이터를 활용해 모델을 학습시키는 방법
- 학습된 모델을 사용하여 레이블이 없는 데이터에 대한 예측을 수행하고, 예측 결과 중 신뢰할 수 있는 일부 데이터를 레이블링에 추가로 사용하여 새로운 학습용 데이터를 생성
> 해당 과정을 반복하여 모델을 업데이트하면서 점진적으로 더 많은 레이블을 확보해나가는 방식

In [13]:
from sklearn.semi_supervised import SelfTrainingClassifier

# Self Training 분류기 생성
self_training_model = SelfTrainingClassifier(base_model)

In [14]:
### Self Training을 위한 함수

def run_self_training(self_training_model, X_labeled, y_labeled,
                      X_unlabeled, early_stopping_rounds = 5):
    iter = 0 # 반복 횟수 카운트를 위한 변수
    rounds_without_improvement = 0 # early stopping을 위한 카운트 변수

    while X_unlabeled.shape[0] > 0 and rounds_without_improvement < early_stopping_rounds:
        iter = iter + 1
        print("=== Iteration {} ===".format(iter))

        ## 1. 레이블이 있는(labeled) 데이터로 모델 학습
        self_training_model.fit(X_labeled, y_labeled)

        ## 2. 훈련된 모델로 레이블이 없는 데이터를 분류
        y_pred_unlabeled = self_training_model.predict(X_unlabeled)

        ## 3. 의사 레이블(pseudo label) 설정
        # 모델이 90% 이상 확신하는 데이터를 labeled 데이터에 추가
        confident_mask = self_training_model.predict_proba(X_unlabeled).max(axis = 1) > 0.9
        X_confident = X_unlabeled[confident_mask]
        y_confident = y_pred_unlabeled[confident_mask]
        # 신뢰할 수 있는 데이터를 레이블링에 추가하여 새로운 학습용 데이터 구성
        # 레이블링 된 데이터의 수를 증가시키는 과정
        X_labeled = np.concatenate([X_labeled, X_confident], axis = 0)
        y_labeled = np.concatenate([y_labeled, y_confident], axis = 0)
        # 의사 레이블링 된 데이터들은 이제 레이블이 없는 데이터에서 제거
        X_unlabeled = X_unlabeled[~confident_mask]

        print("{}개 레이블링 완료".format(X_confident.shape[0]))

        ## Early Stopping 체크
        if X_confident.shape[0] <= 30:
            rounds_without_improvement += 1
            print("Early Stopping 카운트: {}/{}".format(rounds_without_improvement,early_stopping_rounds))
            print()
        else:
            rounds_without_improvement = 0
            print()
    print()
    print("레이블링 완료")

    # 라벨링 된 데이터 반환
    return X_labeled, y_labeled, X_unlabeled

In [15]:
# Self-training을 통한 레이블링 수행

X_labeled, y_labeled, X_unlabeled = run_self_training(base_model, X_labeled, y_labeled, 
                                                      X_unlabeled, early_stopping_rounds = 10)

=== Iteration 1 ===
76730개 레이블링 완료

=== Iteration 2 ===
52198개 레이블링 완료

=== Iteration 3 ===
39200개 레이블링 완료

=== Iteration 4 ===
29764개 레이블링 완료

=== Iteration 5 ===
20522개 레이블링 완료

=== Iteration 6 ===
16465개 레이블링 완료

=== Iteration 7 ===
15364개 레이블링 완료

=== Iteration 8 ===
14153개 레이블링 완료

=== Iteration 9 ===
11024개 레이블링 완료

=== Iteration 10 ===
8405개 레이블링 완료

=== Iteration 11 ===
5797개 레이블링 완료

=== Iteration 12 ===
5002개 레이블링 완료

=== Iteration 13 ===
5057개 레이블링 완료

=== Iteration 14 ===
5340개 레이블링 완료

=== Iteration 15 ===
5964개 레이블링 완료

=== Iteration 16 ===
4986개 레이블링 완료

=== Iteration 17 ===
3506개 레이블링 완료

=== Iteration 18 ===
2778개 레이블링 완료

=== Iteration 19 ===
2924개 레이블링 완료

=== Iteration 20 ===
2593개 레이블링 완료

=== Iteration 21 ===
2516개 레이블링 완료

=== Iteration 22 ===
2430개 레이블링 완료

=== Iteration 23 ===
2090개 레이블링 완료

=== Iteration 24 ===
2193개 레이블링 완료

=== Iteration 25 ===
2031개 레이블링 완료

=== Iteration 26 ===
1540개 레이블링 완료

=== Iteration 27 ===
1065개 레이블링 완료

=== Iteration 28 ===
880개 레이

In [16]:
print(X_labeled.shape[0])
print(X_unlabeled.shape[0]/(X_labeled.shape[0] + X_unlabeled.shape[0]))

526763
0.2196277119767709


- 라벨링 된 데이터의 개수가 172245에서 526763로 증가
- 아직 약 22%의 데이터가 라벨링되지 않은 상황

In [17]:
### 최종 데이터 병합
# 라벨링 된 데이터만 활용

X_train_final = X_labeled
y_train_final = y_labeled

# **4. 최종 예측을 위한 모델링(Final Model Modeling)**

### **📚 결과 정리**

|Model|LogLoss|
|-------|-----|
|DecisionTreeClassifier|0.199358|
|RandomForest|0.197863|
|GradientBoosting|0.193088|
|LightGBM|0.192001| 
|XGBoost|0.192232| 

=> 최종 Model로 **LightGBM**을 채택함

In [29]:
sample_submission.head(3)

,Not_Delayed,Delayed
ID,,
TEST_000000,0,1
TEST_000001,0,1
TEST_000002,0,1


- Delayed = 1
- Not_Delayed = 0

In [27]:
from sklearn.model_selection import train_test_split

X_train,X_valid,y_train,y_valid = train_test_split(X_train_final,y_train_final, test_size = 0.2, random_state = 156)

In [41]:
### 최종 모델 생성
final_classifier = LGBMClassifier(n_estimators = 200, num_leaves = 32, subsample = 0.8,
                                   min_child_samples = 100, max_depth = 128)

# early_stopping_rounds와 eval_metric을 설정하여 모델 학습
evals = [(X_valid, y_valid)]
final_classifier.fit(X_train, y_train, early_stopping_rounds = 100, eval_metric = "logloss",
                      eval_set = evals, verbose = 0)

LGBMClassifier(max_depth=128, min_child_samples=100, n_estimators=200,
               num_leaves=32, subsample=0.8)

In [42]:
# 최종 예측

preds = final_classifier.predict_proba(X_test) 

In [43]:
preds

array([[0.03873092, 0.96126908],
       [0.06411758, 0.93588242],
       [0.27000891, 0.72999109],
       ...,
       [0.29213298, 0.70786702],
       [0.13504985, 0.86495015],
       [0.03948217, 0.96051783]])

In [44]:
submission = pd.DataFrame(data = preds, columns = sample_submission.columns,index = sample_submission.index)

In [45]:
submission.head(3)

,Not_Delayed,Delayed
ID,,
TEST_000000,0.038731,0.961269
TEST_000001,0.064118,0.935882
TEST_000002,0.270009,0.729991


In [46]:
submission.to_csv('submission_final.csv', index = True)

리더보드 점수
- public: 1.0843793707
- private: 1.7480220651